In [2]:
import pandas as pd
from tqdm import tqdm
import re

In [3]:
results = pd.read_html('https://sport.mkez.tw/games/tt_1131112/front/schedule_ball_score.php')[0]

In [4]:
for i in tqdm(range(1,150)):
    results_temp = pd.read_html('https://sport.mkez.tw/games/tt_1131112/front/schedule_ball_score.php?pageNum_RS=' + str(i))[0]
    #print(html_tables_temp)
    results = pd.concat([results, results_temp])

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [01:00<00:00,  2.48it/s]


In [9]:
results.columns = ['matchID', 'Time', 'Group', 'inGroup_matchID', 'Player1', 'Score', 'Player2', 'Ranking']

In [10]:
results = results.reset_index(drop = True)

In [11]:
results

,matchID,Time,Group,inGroup_matchID,Player1,Score,Player2,Ranking
0,5047,(六) 11:58,女子組第四次賽 圖,#95,(#85勝)合作金庫 - 王意如 勝,3 : 2,(#86勝)臺灣體大 - 張孟媫 70,NaN
1,5096,(六) 11:58,男子組第五次賽 圖,#36,第一銀行 - 楊皓任 125,2 : 3,松山家商 - 何勝恩 勝,NaN
2,5098,(六) 11:57,男子組第五次賽 圖,#38,東泰高中 - 陳俊侑 43,0 : 3,福誠高中 - 張先騅 勝,NaN
3,5100,(六) 11:57,男子組第五次賽 圖,#40,乒乓彭 - 彭王維 勝,3 : 0,第一銀行 - 李俊霆 91,NaN
4,5118,(六) 11:55,男子組第五次賽 圖,#58,臺灣師大 - 邱祐宏(請假) 47,請假 : 0,第一銀行 - 方詠恩 勝,NaN
...,...,...,...,...,...,...,...,...
1007,1007,(四) 05:08,男子組第一次賽 圖,#7,輪空,空 : 0,合作金庫 - 莊忠諺 勝,NaN
1008,1006,(四) 05:08,男子組第一次賽 圖,#6,第一銀行 - 林晉霆 勝,0 : 空,輪空,NaN
1009,1004,(四) 05:07,男子組第一次賽 圖,#4,香山高中 - 何少汯 勝,0 : 空,輪空,NaN
1010,1005,(四) 05:07,男子組第一次賽 圖,#5,輪空,空 : 0,國立體大 - 梁庭瑋 勝,NaN


In [12]:
invalid_scores = ['傷', '空', '請', '棄', '-2', '-5']

In [13]:
for i in invalid_scores:
    f = results['Score'].str.contains(i)
    results = results[~f]

In [15]:
results['Sex'] = results['Group'].str.replace("第.次賽 圖", "", regex = True)

In [16]:
results

,matchID,Time,Group,inGroup_matchID,Player1,Score,Player2,Ranking,Sex
0,5047,(六) 11:58,女子組第四次賽 圖,#95,(#85勝)合作金庫 - 王意如 勝,3 : 2,(#86勝)臺灣體大 - 張孟媫 70,NaN,女子組
1,5096,(六) 11:58,男子組第五次賽 圖,#36,第一銀行 - 楊皓任 125,2 : 3,松山家商 - 何勝恩 勝,NaN,男子組
2,5098,(六) 11:57,男子組第五次賽 圖,#38,東泰高中 - 陳俊侑 43,0 : 3,福誠高中 - 張先騅 勝,NaN,男子組
3,5100,(六) 11:57,男子組第五次賽 圖,#40,乒乓彭 - 彭王維 勝,3 : 0,第一銀行 - 李俊霆 91,NaN,男子組
5,5101,(六) 11:54,男子組第五次賽 圖,#41,東山高中 - 謝承翰 89,0 : 3,合作金庫 - 楊子儀 勝,NaN,男子組
...,...,...,...,...,...,...,...,...,...
948,1034,(二) 10:02,男子組第一次賽 圖,#34,東山高中 - 陳浩洋 80,0 : 3,松山家商 - 何勝恩 勝,NaN,男子組
949,1038,(二) 10:01,男子組第一次賽 圖,#38,第一銀行 - 陳宜樂 49,0 : 3,八馬國際 - 賴竑亦 勝,NaN,男子組
950,1043,(二) 10:01,男子組第一次賽 圖,#43,八馬國際 - 蕭惟之 121,0 : 3,東泰高中 - 陳俊侑 勝,NaN,男子組
952,1036,(二) 09:57,男子組第一次賽 圖,#36,(#2勝)金石乒乓 - 蔡宗桓 113,0 : 3,第一銀行 - 陳奕騰 勝,NaN,男子組


In [34]:
player_stats = {}
for i in results.index:
    #player 1
    Player_entry = re.sub("(\\(.+\\)|[0-9]|勝)", "", results.Player1[i])
    Player_name = Player_entry.split(' - ')[1].strip() + ' - ' + Player_entry.split(' - ')[0]

    
    score = results.Score[i].split(' : ')
    games_W = int(score[0])
    games_L = int(score[1])
    
    if games_W == 3:
        match_W = 1
        match_L = 0
    else:
        match_W = 0
        match_L = 1


    if Player_name not in player_stats.keys():
        player_stats[Player_name] = {'name': Player_name, 'Sex': results.Sex[i],
                                     'matches_W': match_W, 'matches_L': match_L,
                                     'games_W': games_W, 'games_L': games_L,                                    
                                    }
    else:
        player_stats[Player_name]['matches_W'] += match_W
        player_stats[Player_name]['matches_L'] += match_L
        player_stats[Player_name]['games_W'] += games_W
        player_stats[Player_name]['games_L'] += games_L

    #player 2
    Player_entry = re.sub("(\\(.+\\)|[0-9]|勝)", "", results.Player2[i])
    Player_name = Player_entry.split(' - ')[1].strip() + ' - ' + Player_entry.split(' - ')[0]
    score = results.Score[i].split(' : ')
    games_W = int(score[1])
    games_L = int(score[0])

    if games_W == 3:
        match_W = 1
        match_L = 0
    else:
        match_W = 0
        match_L = 1
    
    if Player_name not in player_stats.keys():
        player_stats[Player_name] = {'name': Player_name, 'Sex': results.Sex[i],
                                     'matches_W': match_W, 'matches_L': match_L,
                                     'games_W': games_W, 'games_L': games_L,                                    
                                    }
    else:
        player_stats[Player_name]['matches_W'] += match_W
        player_stats[Player_name]['matches_L'] += match_L
        player_stats[Player_name]['games_W'] += games_W
        player_stats[Player_name]['games_L'] += games_L


In [35]:
player_stats

{'王意如 - 合作金庫': {'name': '王意如 - 合作金庫',
  'Sex': '女子組',
  'matches_W': 8,
  'matches_L': 3,
  'games_W': 28,
  'games_L': 17},
 '張孟媫 - 臺灣體大': {'name': '張孟媫 - 臺灣體大',
  'Sex': '女子組',
  'matches_W': 4,
  'matches_L': 3,
  'games_W': 15,
  'games_L': 11},
 '楊皓任 - 第一銀行': {'name': '楊皓任 - 第一銀行',
  'Sex': '男子組',
  'matches_W': 9,
  'matches_L': 5,
  'games_W': 33,
  'games_L': 20},
 '何恩 - 松山家商': {'name': '何恩 - 松山家商',
  'Sex': '男子組',
  'matches_W': 7,
  'matches_L': 4,
  'games_W': 24,
  'games_L': 17},
 '陳俊侑 - 東泰高中': {'name': '陳俊侑 - 東泰高中',
  'Sex': '男子組',
  'matches_W': 3,
  'matches_L': 5,
  'games_W': 12,
  'games_L': 15},
 '張先騅 - 福誠高中': {'name': '張先騅 - 福誠高中',
  'Sex': '男子組',
  'matches_W': 3,
  'matches_L': 4,
  'games_W': 14,
  'games_L': 14},
 '彭王維 - 乒乓彭': {'name': '彭王維 - 乒乓彭',
  'Sex': '男子組',
  'matches_W': 10,
  'matches_L': 4,
  'games_W': 34,
  'games_L': 19},
 '李俊霆 - 第一銀行': {'name': '李俊霆 - 第一銀行',
  'Sex': '男子組',
  'matches_W': 1,
  'matches_L': 5,
  'games_W': 5,
  'games_L': 15},
 '謝承

In [36]:
player_stats = pd.DataFrame.from_dict(player_stats, orient='index').reset_index(drop = True)

In [61]:
selected_1st_rd = ['吳映萱 - 國泰人壽', '葉伊恬 - 合作金庫', '洪敬愷 - 第一銀行', '蘇昱綸 - 合作金庫']
selected_2nd_rd = ['彭郁涵 - 國泰人壽', '李昱諄 - 國泰人壽', '黎彥君 - 第一銀行', '張佑安 - 第一銀行']
selected_3rd_rd = ['林彥均 - 第一銀行', '黃彥誠 - 合作金庫', '陳忞昕 - 合作金庫', '黃禹喬 - 國泰人壽']

selected = [selected_1st_rd, selected_2nd_rd, selected_3rd_rd]

player_stats['selection'] = ''
for i in range(len(selected)):
    print(selected[i])
    player_stats.loc[player_stats['name'].isin(selected[i]), 'selection'] = '第' + str(i+1) + '次賽入選'

['吳映萱 - 國泰人壽', '葉伊恬 - 合作金庫', '洪敬愷 - 第一銀行', '蘇昱綸 - 合作金庫']
['彭郁涵 - 國泰人壽', '李昱諄 - 國泰人壽', '黎彥君 - 第一銀行', '張佑安 - 第一銀行']
['林彥均 - 第一銀行', '黃彥誠 - 合作金庫', '陳忞昕 - 合作金庫', '黃禹喬 - 國泰人壽']


In [62]:
player_stats

,name,Sex,matches_W,matches_L,games_W,games_L,selection,matches_Total,matches_W_pct,games_Total,games_W_pct
222,葉伊恬 - 合作金庫,女子組,6,0,18,3,第1次賽入選,6,1.0,21,0.857
221,蘇昱綸 - 合作金庫,男子組,6,0,18,4,第1次賽入選,6,1.0,22,0.818
220,吳映萱 - 國泰人壽,女子組,6,0,18,5,第1次賽入選,6,1.0,23,0.783
219,洪敬愷 - 第一銀行,男子組,6,0,18,7,第1次賽入選,6,1.0,25,0.720
216,黎彥君 - 第一銀行,男子組,9,1,28,10,第2次賽入選,10,0.9,38,0.737
...,...,...,...,...,...,...,...,...,...,...,...
76,李冠炫 - 香山高中,男子組,0,5,0,15,,5,0.0,15,0.000
125,張滋育 - 海山高中,女子組,0,4,0,12,,4,0.0,12,0.000
161,蔡可萱 - 中港高中,女子組,0,4,0,12,,4,0.0,12,0.000
172,姚力榳 - Dreams乒乓,女子組,0,4,0,12,,4,0.0,12,0.000


In [63]:
player_stats['matches_Total'] = player_stats.matches_W + player_stats.matches_L
player_stats['matches_W_pct'] = round(player_stats.matches_W / player_stats.matches_Total, 3)
player_stats['games_Total'] = player_stats.games_W + player_stats.games_L
player_stats['games_W_pct'] = round(player_stats.games_W / player_stats.games_Total, 3)

In [64]:
player_stats = player_stats.sort_values(by=['matches_W_pct', 'games_W_pct'], ascending = False)

In [65]:
player_stats.to_csv('114_results.tsv', sep="\t", index = False) 

player_stats_f = player_stats[player_stats['Sex'].str.contains('女')]
player_stats_f.to_csv('114_results_f.tsv', sep="\t", index = False) 

player_stats_m = player_stats[player_stats['Sex'].str.contains('男')]
player_stats_m.to_csv('114_results_m.tsv', sep="\t", index = False) 